In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys,os
sys.path.append('./sailfish/')


In [ ]:
!pwd

In [ ]:
%%writefile ex1.py
import numpy as np
from sailfish.subdomain import Subdomain2D
from sailfish.node_type import NTFullBBWall, NTEquilibriumVelocity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim

class MyBlock(Subdomain2D):
    max_v = 0.1

    def boundary_conditions(self, hx, hy):
        wall_map = ( (hx == self.gx-1) | (hx == 0) | (hy == 0) )
        self.set_node( (hy == self.gy-1) & (hx>0) & (hx<self.gx-1) , NTEquilibriumVelocity((self.max_v, 0.0)) )
        self.set_node(wall_map, NTFullBBWall)
        
    def initial_conditions(self, sim, hx, hy):
        sim.rho[:] = 1.0

class MySim(LBFluidSim):
    subdomain = MyBlock

LBSimulationController(MySim).run()

In [ ]:
%%sh
export PYTHONPATH=$PWD/sailfish/
python ex1.py --max_iters=100 --every=100 \
 --lat_nx=30 --lat_ny=30 \
 --visc=0.1\
 --output=ex1 --output_format=npy --debug_dump_dists --block_size=32

In [ ]:
!ls -lrta ex1*

In [ ]:
data = np.load("ex1_dists.0.099.npz")
data['arr_0'][0].shape

In [ ]:
data['arr_0'].shape

In [ ]:
data.files

In [ ]:
dists = data['arr_0']
ny,nx = 32,32
Y,X = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(5,ny/nx*5))
plt.xlim(1,nx-2)
plt.ylim(1,ny-2)
plt.contourf(X,Y,dists[8,:,:],256)


In [ ]:
data = np.load("ex1.0.100.npz")
vx,vy = data['v']
ny,nx = vx.shape
Y,X = np.mgrid[0:ny,0:nx]
plt.figure(figsize=(5,ny/nx*5))
plt.xlim(0,nx-1)
plt.ylim(0,ny-1)
plt.contourf(X,Y,np.sqrt(vy**2+vx**2),256)
plt.streamplot(X,Y,vx,vy,color='white')

In [ ]:
vx[27,15],dists[:,27,15]


In [ ]:
np.sum(dists[:,27,15])

In [ ]:
from sailfish import sym
from sailfish import sym_equilibrium
from sympy import Matrix



In [ ]:
sym.D2Q9.basis

In [ ]:
sum([map(float,c*fi)[0] for c,fi in zip(sym.D2Q9.basis,dists[:,28,16])])/np.sum(dists[:,27,15])

In [ ]:
dists[:,28,16]

In [ ]:
sym_equilibrium.bgk_equilibrium??

In [ ]:
class C:
    incompressible = False
    minimize_roundoff = False
config = C()
sym_equilibrium.bgk_equilibrium(sym.D2Q9,config)

In [ ]:
import sympy as S
g0m0,g0m1x,g0m1y = S.symbols('g0m0,g0m1x,g0m1y')

sym_equilibrium.bgk_equilibrium(sym.D2Q9,config)[0][0].subs([(g0m0,1),(g0m1x,0),(g0m1y,0)]).n()

In [ ]:
sym_equilibrium.bgk_equilibrium(sym.D2Q9,config)[0][1]

In [ ]:
plt.figure(figsize=(6,6))
ax = plt.axes()

for b in sym.D2Q9.basis[1:]:
    ax.arrow(0, 0, *map(float,b), head_width=0.05, head_length=0.1, fc='k', ec='k')

plt.xlim(-1.2,1.2)
plt.ylim(-1.2,1.2)

In [ ]:
eq_fncs = sym_equilibrium.bgk_equilibrium(sym.D2Q9,config)[0]

In [ ]:
d = [float(ex.subs([(g0m0,1),(g0m1x,.2),(g0m1y,0.2)]) ) for ex in eq_fncs]
d

In [ ]:
plt.figure(figsize=(6,6))
ax = plt.axes()

for w,b in zip(sym.D2Q9.basis[1:],d[1:]):
    ax.arrow(0, 0, *map(float,b*w.normalized()), head_width=0.02, head_length=0.03, fc='r', ec='k')
a=.4
plt.xlim(-a,a)
plt.ylim(-a,a)

In [ ]:
d = [float(ex.subs([(g0m0,1),(g0m1x,vx[27,15]),(g0m1y,vy[27,15])]) ) for ex in eq_fncs]
d

In [ ]:
d = [float(ex.subs([(g0m0,1),(g0m1x,vx[27,15]),(g0m1y,vy[27,15])]) ) for ex in eq_fncs]
dnon = dists[:,28,16]
plt.figure(figsize=(6,6))
ax = plt.axes()

for w,b in zip(sym.D2Q9.basis[1:],d[1:]):
    ax.arrow(0, 0, *map(float,b*w.normalized()), head_width=0.02, head_length=0.03, fc='r', ec='k')
for w,b in zip(sym.D2Q9.basis[1:],dnon[1:]):
    ax.arrow(0, 0, *map(float,b*w.normalized()), head_width=0.01, head_length=0.02, fc='b', ec='b')
    
a=.3
plt.xlim(-a,a)
plt.ylim(-a,a)

In [ ]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import clear_output, display, HTML


def show_eq(vx,vy):
    
    d = [float(ex.subs([(g0m0,1),(g0m1x,vx),(g0m1y,vy)]) ) for ex in eq_fncs]

    plt.figure(figsize=(6,6))
    ax = plt.axes()

    for w,b in zip(sym.D2Q9.basis[1:],d[1:]):
        ax.arrow(0, 0, *map(float,b*w.normalized()), head_width=0.02, head_length=0.03, fc='r', ec='k')
        if b<0:
            print "Negative distribution:",w
    a=.4
    plt.xlim(-a,a)
    plt.ylim(-a,a)
    
    plt.show()

In [ ]:
w = interactive(show_eq, vx=(0.,1.0), vy=(0,1.))
display(w)